In this notebook we would like to exemplify how the Scattering features of an image allow to classify 
correctly textures, obtaining a better performance than other methods features obtained using 
the mean of wavelet-filtered images.

As a reference we use the example at the scikit-image web page that uses Gabor filters for classifying 
three different textures. We will show, that for those 3 textures, these Gabor coefficients perform very well,
but as we augment the number of textures, the problem get more difficult, and the Gabor coefficients start not
performing that well. This is not the case with the Scattering features which still perform correctly 
(~90% correctly classified).

The database used is a subset of the well know Broadtz texture database, which is online and well known as a
benchmark in image classification. 

The Scattering features that we extract in this notebook are a simplification of the proposed features in 

Mallat, S. 'Scattering transform'

More specifically, this coefficients are just the first layer of the Scattering transform. 
For more complicated classification 
problems, such as general image classification (CIFAR, or ImageNet), more advanced features are needed.
For these cases, Scattering features have shown to be useful, as reported in:
    
    - Bruna, Mallat. 'Image Classification'
    - Oyallon, Mallat. 'More image classification'
    


In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
from scipy import ndimage as ndi
import skimage 
from skimage import data
from skimage.util import img_as_float
from skimage.filters import gabor_kernel


In [2]:
#Similarly to the example in scikit-image for Gabor filters, here we construct a Gabor filterbank
# with J scales and L angles
def create_gabor_filterbank(J=4,L=8):
    kernels =[]
    for scale in 2 ** np.arange(J) * .1:
        filter_scale = []
        kernels.append(filter_scale)
        for theta in np.arange(L) / float(L) * np.pi:
            gabor = gabor_kernel(scale, theta=theta)
            filter_scale.append(gabor)
    return kernels

In [5]:
#We create the set of Gabor filters
filters = create_gabor_filterbank(J=6,L=8)

In [6]:
#Following the exemple, we load just three textures 
#extract exemplar data

from sklearn.feature_extraction.image import extract_patches_2d

shrink = (slice(0, None, 1), slice(0, None, 1))
image_names = ('brick', 'grass', 'wall')
brick = img_as_float(data.load('brick.png'))[shrink]
grass = img_as_float(data.load('grass.png'))[shrink]
wall = img_as_float(data.load('rough-wall.png'))[shrink]

images = (brick, grass, wall)

px = 64

brick_patches, grass_patches,wall_patches=(extract_patches_2d(image,[px, px],max_patches=100) for image in images)

all_patches = np.concatenate([brick_patches, grass_patches,wall_patches],axis=0)
all_labels = np.arange(300)//100


In [7]:
#This function computes the 'Gabor-like' features presented in the examples of the scikit-image web page. 
def compute_features_example(images, filters):
    #note that the filters have different sizes, thus easier to apply the con
    # in the spatial domain

    feats = np.zeros((len(images), len(filters), len(filters[0]),2), dtype=np.float32)
    for i,image in enumerate(images):
        image_features=feats[i] #pointer
        for scale,scale_output in zip(filters,image_features):
            for kernel,kernel_l_output in zip(scale,scale_output):
                filtered = ndi.convolve(image, kernel, mode='wrap')
                kernel_l_output[:] = filtered.mean(),filtered.var()

    return feats

In [8]:
#This function computes the first layer of the scattering transform, for each image. This amounts to compute: 
# -convolve the image with the gabor filters (psi): u=conv(x,psi)
# -apply a non-linearity, more specifically, the modulus:  v=abs(u)
# -mean and variance of the values: output= mean(v), mean(v^2)
# The second layer of the scattering transform applies again the same sequence of operations. This is left as
# future work.

def compute_scattering_layer1(images, filters):
    #note that the filters have different sizes, thus easier to apply the con
    # in the spatial domain

    feats = np.zeros((len(images), len(filters), len(filters[0]),2), dtype=np.float32)
    for i,image in enumerate(images):
        image_features=feats[i] #pointer
        for scale,scale_output in zip(filters,image_features):
            for kernel,kernel_l_output in zip(scale,scale_output):
                
                # |x * psi| * phi
                filtered = np.abs(ndi.convolve(image, kernel, mode='wrap'))
                kernel_l_output[:] = filtered.mean(), (filtered ** 2).mean()

    return feats

In [9]:
scat_features = compute_scattering_layer1(all_patches,filters)
ex_features = compute_features_example(all_patches,filters)

scat_matrix = scat_features.reshape((len(scat_features),-1))
ex_matrix = ex_features.reshape((len(ex_features),-1))
patches_matrix = all_patches.reshape((len(all_patches),-1))


/Users/ferradans/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:474: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [ ]:
#Function that computes the classification scores, given a set of features and labels
#using standard linear regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score, KFold, ShuffleSplit

def from_features_to_classif_scores(features,labels):
    #stack the features for learning
    features = features.reshape((len(features),-1))
    # apply learning pipeline
    n = len(features)
    pipeline = make_pipeline(Normalizer(),StandardScaler(),LogisticRegression(C=1.0))
    cv = ShuffleSplit(n,n_iter=3,test_size=1, train_size=1)
    
    scores = cross_val_score(pipeline,features,labels,cv=5,n_jobs=5)
    print('score:',scores)


In [ ]:
#Now that we have the features, we want to see how the Gabor features and scattering coefficients
#allow a correct classification, but not the image-patches themselves.
print('Example initial BD:')
from_features_to_classif_scores(ex_features,all_labels)

print('Scat initial BD:')
from_features_to_classif_scores(scat_features,all_labels)

print('Patches initial BD:')
from_features_to_classif_scores(all_patches,all_labels)

print('We can see that the scattering features and the gabor-like features give perfect classification ratings')


In [ ]:
#The previous problem is to easy to evaluate the gabor and scattering features. 
#Thus we propose to use a more complex database
# Here, we load the new database and preprocess by extracting patches and extracting the mean

import scipy.misc
import glob
files = glob.glob('./textures/*.tiff')
broadtz_patches  = []
broadtz_labels = []
view_images = bool(1)
if view_images: 
    fig, axes = plt.subplots(nrows=10, ncols=6, figsize=(30, 30))
    AX = axes.ravel()
for i,file in enumerate(files):
    aa = data.imread(file)
  
    #possibly need a preprocessing (re-scaling)
    aa = aa[:256,:256]
    aa.shape
    broadtz_patches.append(extract_patches_2d(aa,[px, px],max_patches=50))
    broadtz_labels.append([i]*len(broadtz_patches[-1]))
    if view_images :
        AX[i].imshow(aa, cmap='gray')
        AX[i].axis('off')
    
broadtz_labels = np.concatenate(broadtz_labels, axis=0)
broadtz_patches= np.concatenate(broadtz_patches,axis=0)

#take out the mean of each patch: we want to be indep. from illumination changes
#preprocessing
brodatz_means = broadtz_patches.reshape(len(broadtz_patches), -1).mean(1).astype('float32')
broadtz_patches = broadtz_patches - brodatz_means[:, np.newaxis, np.newaxis]


Finally, we obtain the classification results on the new data set, using the gabor-like and scattering features. 
We observe that the scattering features outperform in almost 30% the gabor-like features.

In [ ]:
#From the patches extrated from the new database, we extract the different features
scat_features_b=compute_scattering_layer1(broadtz_patches,filters)
ex_features_b=compute_features_example(broadtz_patches,filters)

print('Example initial BD:') 
from_features_to_classif_scores(ex_features_b,broadtz_labels)
print('Scat initial BD:') 
from_features_to_classif_scores(scat_features_b,broadtz_labels)